In [1]:
import syft as sy
duet = sy.duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!

`searchable` is deprecated please use `pointable` in futurees: 4  Request Handlers: 1                                


In [2]:
duet.requests.add_handler(action="accept")

In [3]:
duet.store.pandas

""


In [4]:
from torchvision import datasets
from torchvision import transforms

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('mnist', download=True, train=True, transform=transform)
# valset = datasets.MNIST('mnist', download=True, train=False, transform=transform)


data = trainset.data.view(trainset.data.shape[0], -1).float()
data = data[:5000]
data.shape

torch.Size([5000, 784])

In [5]:
data.send(duet, searchable=True, tags=["train_mnist"], description="data")

In [6]:
data.shape

torch.Size([5000, 784])

In [7]:
import torch 
dataloader = torch.utils.data.DataLoader(data, batch_size=128)

labels = trainset.targets[:5000]
targetloader = torch.utils.data.DataLoader(labels, batch_size=128)

In [8]:
from torch import nn
import syft as sy

input_size = 784
hidden_sizes = [256, 500]
output_size = 10

class SyNet_client(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_client, self).__init__(torch_ref=torch_ref)
        self.lin = nn.Linear(input_size, hidden_sizes[0])
        
    def forward(self, x):
        x = self.lin(x)
        x = self.torch_ref.nn.functional.relu(x)
        return x
    
class SyNet_server(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_server, self).__init__(torch_ref=torch_ref)
        self.lin2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.lin3 = nn.Linear(hidden_sizes[1], output_size)
        self.sft = nn.LogSoftmax(dim=1) 
        
    def forward(self, x):
        x = self.lin2(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.lin3(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.sft(x)
        return x

In [57]:
#Model 1
model1 = SyNet_client(torch)
# model1_ptr = model1.send(duet)
opt1 = torch.optim.SGD(params=model1.parameters(),lr=0.03)

#Model 2
model2 = SyNet_server(torch)
opt2 = torch.optim.SGD(params=model2.parameters(),lr=0.03)

In [58]:
for images, target in zip(dataloader, targetloader):    
    opt1.zero_grad()
    opt2.zero_grad()
    
    pred = model2(model1(images))
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
    loss.backward()
        
    opt1.step()
    opt2.step()
    print(loss)
    
#     print(activation)

tensor(8.6705, grad_fn=<NllLossBackward>)
tensor(941.7748, grad_fn=<NllLossBackward>)
tensor(19460.0645, grad_fn=<NllLossBackward>)
tensor(28833.9766, grad_fn=<NllLossBackward>)
tensor(120202.1016, grad_fn=<NllLossBackward>)
tensor(313913.8438, grad_fn=<NllLossBackward>)
tensor(2119398.7500, grad_fn=<NllLossBackward>)
tensor(1066598.1250, grad_fn=<NllLossBackward>)
tensor(139424.5156, grad_fn=<NllLossBackward>)
tensor(13.6937, grad_fn=<NllLossBackward>)
tensor(3.7452, grad_fn=<NllLossBackward>)
tensor(3.8513, grad_fn=<NllLossBackward>)
tensor(30.8226, grad_fn=<NllLossBackward>)
tensor(154623.7500, grad_fn=<NllLossBackward>)
tensor(2.5973e+09, grad_fn=<NllLossBackward>)
tensor(373656.7188, grad_fn=<NllLossBackward>)
tensor(688.9954, grad_fn=<NllLossBackward>)
tensor(281.7310, grad_fn=<NllLossBackward>)
tensor(72313.5156, grad_fn=<NllLossBackward>)
tensor(149.3159, grad_fn=<NllLossBackward>)
tensor(31.1291, grad_fn=<NllLossBackward>)
tensor(15.2911, grad_fn=<NllLossBackward>)
tensor(5747